<a href="https://colab.research.google.com/github/arthur-b-martins/AI-Agents-Lamia/blob/main/pratica-2/groq_agent_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 3.8 MB/s eta 0:00:00


In [ ]:
import os  # necessário para trabalhar com o sistema operacional
import re  # necessário para trabalhar com regex (no final do código)

os.environ['GROQ_API_KEY'] = "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"  # define a variável de ambiente com a chave da API do GROQ


In [ ]:
from groq import Groq  # importa a classe Groq da biblioteca groq

client = Groq(  # cria uma instância do cliente Groq
    api_key=os.environ.get("GROQ_API_KEY"),  # usa a chave da API armazenada na variável de ambiente
)

chat_completion = client.chat.completions.create(  # faz uma requisição para gerar uma resposta do modelo de linguagem
    messages=[  # define a lista de mensagens no estilo "chat", iniciando com a mensagem do usuário
        {
            "role": "user",  # indica que a mensagem foi enviada pelo usuário
            "content": "Explain the importance of fast cars",  # conteúdo da mensagem enviada ao modelo, que no caso foi um exemplo
        }
    ],
    model="llama3-70b-8192",  # especifica o modelo a ser usado (neste caso, LLaMA 3 com 70 bilhões de parâmetros)
)

print(chat_completion.choices[0].message.content)  # imprime a resposta gerada pelo modelo (conteúdo da primeira escolha de resposta)


The thrill of the open road and the rush of adrenaline! While not everyone may share the same enthusiasm, there are several reasons why fast cars hold a special place in the hearts of many:

1. **Adrenaline rush**: Fast cars provide an exhilarating experience, releasing a rush of adrenaline and endorphins that can be addictive. The thrill of acceleration, cornering, and braking is unmatched, making it a thrilling experience.
2. **Sense of freedom**: Fast cars offer a sense of liberation, allowing drivers to feel unencumbered by traffic or road constraints. This emotional high can be empowering, making drivers feel more confident and in control.
3. **Innovation and technology**: Fast cars push the boundaries of innovation, driving technological advancements, and improving overall road safety. High-performance vehicles often feature cutting-edge safety features, such as advanced aerodynamics, and improved braking systems.
4. **Social status**: Fast cars are often seen as status symbols, 

Acima foi o primeiro teste da API, para que pudessemos ver o funcionamento do modelo

In [ ]:
class Agent:  # define uma classe chamada Agent, que representa um agente de IA
  def __init__(self, client, system):  # método construtor, executado ao criar uma instância da classe
    self.client = client  # armazena o cliente da API (por exemplo, uma instância de Groq)
    self.system = system  # armazena a mensagem de sistema (instrução inicial para o modelo)
    self.messages = []  # inicializa a lista de mensagens do histórico da conversa
    if self.system is not None:  # se foi passada uma mensagem de sistema...
      self.messages.append({"role": "system", "content": self.system})  # adiciona a mensagem de sistema ao histórico

  def __call__(self, message=""):  # permite que a instância do agente seja chamada como uma função, com uma mensagem opcional
    if message:  # se uma mensagem foi passada...
      self.messages.append({"role": "user", "content": message})  # adiciona a mensagem do usuário ao histórico
    result = self.execute()  # executa a chamada ao modelo e obtém a resposta
    self.messages.append({"role": "assistant", "content": result})  # adiciona a resposta do modelo ao histórico
    return result  # retorna a resposta gerada

  def execute(self):  # método que envia o histórico de mensagens ao modelo e retorna a resposta
    completion = client.chat.completions.create(  # faz a chamada para gerar uma nova resposta do modelo
        messages=self.messages,  # envia o histórico de mensagens
        model="llama3-70b-8192"  # define o modelo usado (LLaMA 3 com 70B parâmetros)
    )
    return completion.choices[0].message.content  # retorna o conteúdo da resposta gerada


A classe do Agente propriamente dita, com as funções principais:
 - init: para inicializar o agente, com a instância do Groq, e com uma mensagem de sistema.
 - call: é quando o Agente será chamado, onde se passa a query do usuário. A única parte que não ficou muito clara para mim é que o apresentador do video deixa como opção a mensagem ser vazia, porém na função agent_loop a mensagem nunca é vazia (next_prompt inicializa com a query do usuário e não recebe "" em nenhum momento).
 - execute: retorna a resposta do modelo.

In [ ]:
# define o prompt de sistema, que vai definir como o modelo irá reagir
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

In [ ]:
def calculate(operation: str) -> float:  # define uma função que recebe uma operação matemática em formato de string
    return eval(operation)  # avalia a string como uma expressão Python e retorna o resultado (perigoso em código real)

# Eu reorganizei este trecho para manter boas práticas de programação
PLANET_MASSES = {  # dicionário com as massas dos planetas do sistema solar, em quilogramas (kg)
    "mercury": 3.285e23,
    "venus": 4.867e24,
    "earth": 5.972e24,
    "mars": 6.39e23,
    "jupiter": 1.898e27,
    "saturn": 5.683e26,
    "uranus": 8.681e25,
    "neptune": 1.024e26
}

def get_planet_mass(planet: str) -> float | None:  # define uma função que retorna a massa de um planeta, se ele existir no dicionário
    return PLANET_MASSES.get(planet.lower())  # busca a massa do planeta (ignora maiúsculas/minúsculas), ou retorna None se não encontrar



In [ ]:
neil_tyson = Agent(client=client, system=system_prompt) # define um agente

In [ ]:
res = neil_tyson("what is the mass of venus?") # armazena em 'res' a resposta do modelo
print(res) # Mostra a resposta

Thought: I need to find the mass of Venus


Aqui é a primeira resposta do nosso agente que pode calcular
e pegar a massa dos planetas, e a primeira resposta dele segue de acordo com o
prompt de sistema, onde determina que primeiro ele deve pensar o que será necessário fazer para resolver o problema.


In [ ]:

res = neil_tyson.messages # armazena em 'res' todas as mensagens
print(res) # mostra as mensagens

[{'role': 'system', 'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the a

In [ ]:
def agent_loop(max, system, query):  # define uma função que executa um loop com o agente, recebendo: número máximo de iterações, sistema (instrução) e pergunta inicial
  agent = Agent(client, system)  # cria uma instância do agente, passando o cliente e a mensagem de sistema
  next_prompt = query  # define a primeira entrada (pergunta) como o prompt inicial
  tools = ['calculate', 'get_planet_mass']  # define as ferramentas disponíveis que o agente pode usar

  for i in range(max):  # executa um loop até o número máximo de iterações
    result = agent(next_prompt)  # envia o prompt atual para o agente e recebe a resposta
    print(result)  # imprime a resposta do agente

    if "PAUSE" in result:  # se a resposta contém a palavra "PAUSE", significa que o agente quer executar uma ação antes de continuar
      action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)  # usa regex para extrair o nome da ferramenta e o argumento
      chosen_tool = action[0][0]  # extrai o nome da ferramenta da tupla encontrada
      arg = action[0][1]  # extrai o argumento fornecido

      if chosen_tool in tools:  # se a ferramenta for permitida...
        tool_response = eval(f"{chosen_tool}('{arg}')")  # executa a função com o argumento
        next_prompt = f"Observation: {tool_response}"  # cria a próxima entrada baseada na resposta da ferramenta

      else:
        next_prompt = "Observation: tool not found"  # se a ferramenta não for reconhecida

      print(next_prompt)  # imprime a observação gerada
      continue  # pula para a próxima iteração do loop

    if "Answer" in result:  # se a resposta contém "Answer", o agente chegou na resposta
      break  # encerra o loop


Esse trecho simula um ciclo de raciocínio iterativo: o agente pensa, pode decidir pausar e chamar uma função externa (no caso uma calculadora ou massa de um planeta), a observação é o retorno da resposta da ferramenta, adicionada como mensagem, e então continua o raciocínio com base nessa observação,
até que o agente encontre a resposta da pergunta inicial, que teoricamente seria feita pelo usuário.



In [ ]:
agent_loop(10, system_prompt, "What is the mass of mercury times 3?") # chamando a função com máximo de 10 iterações,
# com o prompt de sistema definido no inicio (Pensar, Agir, Receber a observação, até obter a resposta)
# e com a query do 'usuário'

Thought: I need to find the mass of Mercury
.

Action: get_planet_mass: mercury
PAUSE
Observation: 3.285e+23
Thought: I need to multiply this by 3

Action: calculate: 3.285e+23 * 3
PAUSE
Observation: 9.854999999999999e+23
Thought: I have the answer
!

Answer: The mass of mercury times 3 is 9.854999999999999e+23.


E por fim temos o resultado do nosso agente, agindo conforme o prompt de sistema, e dando a saída correta para o usuário. Podemos ver todo seu ciclo de pensamento.